# Ideia Geral e Motivação

Depois de nossa aula sobre passeios aleatórios, pensamos em como estimar a esperança de passeios com parêmetros diferentes dos que aprendemos em aula. Relembrando, para passos de 1 e -1, início em $a$, piso em 0 e teto em N, podemos calcular a esperança de duraçã do jogo, $\mu_a$, da seguinte maneira:
$$
E(T_a)=
\begin{cases}
    a(N-a)~~~p=q=1/2\\
    \frac{a}{q-p}-\frac{N}{q-p}(\frac{1-(q/p)^a}{1-(q/p)^N})~~~p\neq q    
\end{cases}
$$
Todavia, essas formulas perdem sua funcionalidade ao trocar qualquer um desses parâmetros, como a quantidade de acréscimo e decréscimo a cada passo.

Sendo assim, buscamos alternativas para estimar a duração desses passeios

## Inspiração na mecânica

Como uma função que media tempo, para nós foi natural buscar nos conceitos da física analogias que nos guiassem a uma estimativa dessa esperança. Sob este prisma, calcularíamos essa duração como uma simples razão entre "distância" e "velocidade".

Antes de prosseguirmos, adotaremos alguns termos, nomenclaturas e definições.
Sejam $i$ o valor inicial do passeio, $N$ o valor teto de parada, $n$ o valor mínimo de parada (ruína), $a$ o acréscimo ou passo adiante dado com probabilidade p e $d$ o decréscimo ou passo para trás dado com probabilidade $q=1-p$,
Seja $\lambda = p \cdot a + q \cdot d$,
Então o estimador para $E(T_a)$, chamado de $\hat{\mu_t}$, é dado por:
$$
\hat{\mu_t} = 
\begin{cases}
    \frac{N-a}{\lambda}, ~~~ \lambda > 0\\
    \frac{n-a}{\lambda}, ~~~ \lambda < 0
\end{cases}
$$

*Observação Interessante:*
 > Note que, quando $lambda = 0$, esse estimador não tem valor determinado. Contudo, supondo que tivesse e assumisse qualquer uma das formas, poderíamos calcular $\lim_{\lambda\rightarrow 0}$, o que nos daria $inf$. Ou seja, intuitivamente, esperar-se-ia que o passeio não tivesse fim ("sobrevivesse o jogo"). Entretanto, sabemos que não é bem assim que funciona a expectativa de sobrevivência do passeio, o que indica uma discrepância para valores de $\lambda$ próximos de 0. Exploraremos isso posteriormente.

## Testagem do estimador

Partimos em seguida para a testagem de $\hat{\mu_t}$ em busca de padrões de inconsistência para então desenvolvermos alguma forma de correção ou nova abordagem. Para esses testes, usamos inicialmente a linguagem de programação Python. Pouco tempo depois, porém, resolvemos reescrever nosso código na linguagem de programação Julia por sua velocidade e por não necessitarmos a primeiro momento de bibliotecas exclusivas ao ambiente Python.

Comparamos então nossa fórmula com $10^6$ simulações com os mesmos parâmetros. Confira nosso simulador e a função que calcula $\mu_t$:


In [ ]:
function simulador(topo, inicial, piso, p, acr, decr)

    totalsims = 10^6 # Aumente para maior precisão. Em ^6, cada loop de main() demora 0.15s.
    maxits = 10^10
    duracaototal = 0.0

    for i in 1:totalsims
        duracao = 0.0
        valoratual = inicial
        while valoratual > piso && valoratual < topo && duracao < maxits
            duracao += 1.0
            if rand() > p
                valoratual += decr
            else
                valoratual += acr
            end
        end
        duracaototal += duracao
    end
    # Com ou sem arredondamento para cima (considerando como funcionam passeios, pode-se
    # querer um valor inteiro)
    # return ceil(duracaototal / totalsims) 
    return duracaototal / totalsims
end

function estimatempo(topo, inicial, piso, p, acr, decr)

    fator = p * acr + (1 - p) * decr
    return fator > 0 ? (topo - inicial) / fator : abs((inicial - piso) / fator)
end

# Análise dos Resultados
Com as simulações realizadas e as estimações feitas, compilamos os dados em um DataFrame e o salvamos como CSV para uso futuro

In [ ]:
function main()
    totalcomparacoes = 10^4 # DURAÇÃO: ~ 0.15segundos * 10^i (deixe entre 4 e 5)

    dfpasseios = DataFrame(
        :Estimada => Float64[],
        :Simulada => Float64[],
        :Topo => Float64[],
        :Inicial => Float64[],
        :Piso => Float64[],
        :Probabilidade => Float64[],
        :Acrescimo => Float64[],
        :Decrescimo => Float64[]
    )
    if "fromCSV" in ARGS
        dfpasseios = DataFrame(CSV.File("./CSV/dados.csv"))
    else
        difftotal = 0
        for i in 1:totalcomparacoes
            push!(dfpasseios, gera_passeios())
        end
    end

    dfpasseios.:Diferenca = dfpasseios.:Estimada - dfpasseios.:Simulada
    transform!(dfpasseios, :Diferenca => ByRow(abs) => :Diferenca)
    transform!(dfpasseios, :Diferenca => ByRow(ceil) => :Diferenca)

    media = mean(dfpasseios.:Diferenca)
    mediana = median(dfpasseios.:Diferenca)
    println("Média dos erros: $media\n")
    println("Mediana dos erros: $mediana\n")

    if "toCSV" in ARGS
        CSV.write("./CSV/dados.csv", dfpasseios)
    end

    if "graficar" in ARGS
        graficar(dfpasseios)
    end

end

function gera_passeios()
    piso = rand(0.0:10.0)
    topo = rand(piso+2.0:100.0)
    inicial = rand(piso+1.0:topo-1.0)

    acr = rand(1.0:5.0)
    decr = rand(-5.0:-1.0)

    p = 0.0
    while p == 0.0
        p = rand()
    end

    duracaoestimada = estimatempo(topo, inicial, piso, p, acr, decr)
    duracaosimulada = simulador(topo, inicial, piso, p, acr, decr)


    return duracaoestimada, duracaosimulada, topo, inicial, piso, p, acr, decr
end

Com tudo isso preparado, geramos os gráficos iniciais de nossa análise.

In [ ]:
function graficar(dfpasseios)

    x = dfpasseios.:Probabilidade
    distancia = dfpasseios.:Diferenca

    graficos = scatter(x, distancia)

    savefig(graficos, "~/Projetos/Estimador-de-Passeio/Graficos/Grafico.pdf")
end

Confira o gráfico:

![Figura 1.1](./Graficos/Grafico.png)